In [ ]:
import os
import re
import json
import open3d as o3d
import numpy as np
import CSF
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
from scipy.spatial import cKDTree
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1-2*qy*qy-2*qz*qz,   2*qx*qy-2*qz*qw,     2*qx*qz+2*qy*qw],
        [2*qx*qy+2*qz*qw,     1-2*qx*qx-2*qz*qz,   2*qy*qz-2*qx*qw],
        [2*qx*qz-2*qy*qw,     2*qy*qz+2*qx*qw,     1-2*qx*qx-2*qy*qy]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)

def compute_labels_for_frame(track_path, frame_idx, threshold=1.2):
    plyf   = os.path.join(track_path, f"{frame_idx}.ply")
    car0f  = os.path.join(track_path, f"{frame_idx}_car_state.json")
    coll0f = os.path.join(track_path, f"{frame_idx}_collision_info.json")
    cloud = o3d.io.read_point_cloud(plyf)
    pts   = np.asarray(cloud.points)
    car0  = json.load(open(car0f))
    pos0, R0 = get_pose(car0)
    world_pts = (R0 @ pts.T).T + pos0

    ever_close     = np.zeros(len(world_pts), dtype=bool)
    ever_collision = np.zeros(len(world_pts), dtype=bool)
    min_dists = np.ones(len(world_pts)) * np.inf
    risk_score = np.ones(len(world_pts)) * 1.0  # safest default

    j = 1
    while True:
        carjf  = os.path.join(track_path, f"{j}_car_state.json")
        colljf = os.path.join(track_path, f"{j}_collision_info.json")
        if not (os.path.exists(carjf) and os.path.exists(colljf)):
            break

        carj  = json.load(open(carjf))
        collj = json.load(open(colljf))
        posj, _ = get_pose(carj)

        d_car = np.linalg.norm(world_pts - posj, axis=1)
        min_dists = np.minimum(min_dists, d_car)
        ever_close |= (d_car < (threshold - 0.2))

        if collj['has_collided']:
            cp = collj['position']
            pos_col = np.array([cp['x_val'], cp['y_val'], cp['z_val']], dtype=float)
            d_col = np.linalg.norm(world_pts - pos_col, axis=1)
            is_collision = d_col < 1.0
            risk_score[is_collision] = 10.0

        j += 1

    # Compute 10-class safety label
    # normalized = np.clip(min_dists, 0, 5)
    labels = np.ceil((5 - min_dists) / 0.8).astype(int)
    labels[risk_score == 10.0] = 10
    labels = np.clip(labels, 1, 10)
    return labels

# # custom colormap for final visualization
# colors = [(0.5, 0.5, 0.5), (1,1,0), (1,0,0)]
# custom_cmap = LinearSegmentedColormap.from_list("gray_yellow_red", colors, N=10)

# MAIN
data_root = "../data"
output_frame_dir = "rendered_frames"
os.makedirs(output_frame_dir, exist_ok=True)
threshold_check = 1.2

for track_id in range(8):
    track_folder = os.path.join(data_root, str(track_id))
    ply_files = sorted(
        [f for f in os.listdir(track_folder) if re.match(r"^\d+\.ply$", f)],
        key=lambda x: int(x.split('.')[0])
    )

    for ply_name in tqdm(ply_files, desc=f"Track {track_id}"):
        idx = int(ply_name.split('.')[0])

        # 1) load raw points
        pcd = o3d.io.read_point_cloud(os.path.join(track_folder, ply_name))
        xyz = np.asarray(pcd.points)

        # 2) CSF ground vs non-ground
        csf = CSF.CSF()
        csf.params.bSloopSmooth     = False
        csf.params.cloth_resolution = 0.03
        csf.params.rigidness        = 1
        csf.params.iterations       = 500
        csf.params.class_threshold  = 0.1
        csf.setPointCloud(xyz)

        ground_csfi     = CSF.VecInt()
        non_ground_csfi = CSF.VecInt()
        csf.do_filtering(ground_csfi, non_ground_csfi)
        obstacle_idx = np.array(list(non_ground_csfi), dtype=int)
        ground_idx   = np.array(list(ground_csfi),     dtype=int)

        # 3) static-distance labels (1–9 + 10 for obstacles)
        tree = cKDTree(xyz[obstacle_idx])
        dist_to_obs, _ = tree.query(xyz, k=1, workers=-1)
        max_d      = np.percentile(dist_to_obs, 99)
        normalized = np.clip(dist_to_obs / max_d * 9, 0, 9)
        yn_labels  = np.ceil((9 - normalized) / 1).astype(int)
        yn_labels  = np.clip(yn_labels, 1, 9)
        yn_labels[obstacle_idx] = 10

        # 4) dynamic-future labels from compute_labels_for_frame (1–10)
        dyn_labels = compute_labels_for_frame(track_folder, idx)

        # 5) combine per-point:
        #    by default average the two;
        #    but if *either* label is 10, force combined = 10
        combined = (dyn_labels + yn_labels) //2
        mask10   = (dyn_labels == 10) | (yn_labels == 10)
        combined[mask10] = 10
        dyn_only = (dyn_labels == 10) & (yn_labels != 10)
        combined[dyn_only] = np.ceil((dyn_labels[dyn_only] + yn_labels[dyn_only]) / 2).astype(int)

        out_npy = os.path.join(track_folder, f"{idx}_labels_2.npy")
        # print(out_npy)
        np.save(out_npy, combined)

        # 6) visualize combined label
        # norm_comb = (combined - 1) / 9.0        # maps 1→0.0 … 10→1.0
        # rgb       = custom_cmap(norm_comb)[:, :3]

        # vis_pcd = o3d.geometry.PointCloud()
        # vis_pcd.points = o3d.utility.Vector3dVector(xyz)
        # vis_pcd.colors = o3d.utility.Vector3dVector(rgb)

        # o3d.visualization.draw_geometries(
        #     [vis_pcd],
        #     window_name=f"Track {track_id} – Frame {idx}",
        #     width=800, height=600
        # )

visualize pcd

In [ ]:
import open3d as o3d
import numpy as np
import os
import json
from tqdm import tqdm
from scipy.spatial.transform import Rotation as R
import time
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1 - 2*qy**2 - 2*qz**2,   2*qx*qy - 2*qz*qw,     2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw,       1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw,       2*qy*qz + 2*qx*qw,     1 - 2*qx**2 - 2*qy**2]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    rot = quat2rot(o)
    return pos, rot

# KITTI color palette
label_to_color = {
    0: (0.5, 0.5, 0.5),
    1: (0.0, 1.0, 0.0),
    2: (1.0, 0.0, 0.0),
}

# ### CORRECT conversion from your PLY frame (X→right, Y→forward) 
# ###      → KITTI (X→forward, Y→left) is a –90° about Z, not –180°.
R_kitti = R.from_euler('z', -90, degrees=True).as_matrix()

output_frame_dir = "rendered_frames"
# os.makedirs(output_frame_dir, exist_ok=True)

for file_num in tqdm(range(8), desc="Tracks"):
    track_out = os.path.join(output_frame_dir, str(file_num))
    os.makedirs(track_out, exist_ok=True)

    data_dir = f'../data/{file_num}'
    ply_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.ply')],
                       key=lambda x: int(x[:-4]))

    # one window per track
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=1280, height=720, visible=False)

    ctr = vis.get_view_control()
    ctr.set_zoom(0.7)
    for ply_name in tqdm(ply_files, desc=f"Frame in track {file_num}"):
        idx = ply_name[:-4]

        # --- load
        pcd = o3d.io.read_point_cloud(os.path.join(data_dir, ply_name))
        labels = np.load(os.path.join(data_dir, f"{idx}_labels.npy")).astype(int).reshape(-1)
        with open(os.path.join(data_dir, f"{idx}_car_state.json")) as f:
            car_state = json.load(f)

        # --- colorize
        cols = np.array([label_to_color.get(l, label_to_color[0]) for l in labels])
        pcd.colors = o3d.utility.Vector3dVector(cols)


        # --- get pose & flatten to yaw only
        pos, rot_full = get_pose(car_state)
        yaw = np.arctan2(rot_full[1, 0], rot_full[0, 0])
        rot_flat = R.from_euler('z', yaw, degrees=False).as_matrix()

        # --- transform into world with no roll/pitch
        T = np.eye(4)
        T[:3, :3] = rot_flat @ R_kitti
        T[:3, 3] = pos
        # pcd.transform(T)

        # --- chase‐cam in flattened frame
        offset_local = np.array([0, 3, -8.0])
        cam_pos = pos + rot_flat @ (R_kitti @ offset_local)
        lookat = pcd.get_center() + np.array([0, 0, 0.2])

        # --- render
        vis.clear_geometries()
        vis.add_geometry(pcd)
        
        ctr.set_lookat( lookat )
        front = (lookat - cam_pos)
        front /= np.linalg.norm(front)
        ctr.set_front( front.tolist() )
        ctr.set_up([0, 0, 1])
        

        vis.poll_events()
        vis.update_renderer()
        vis.run()
        # time.sleep(0.1)
        img_path = os.path.join(track_out, f"{idx}.png")
        vis.capture_screen_image(img_path)

    vis.destroy_window()


2

In [ ]:
import open3d as o3d
import numpy as np
import os, re
from tqdm import tqdm
import json
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1 - 2*qy**2 - 2*qz**2,   2*qx*qy - 2*qz*qw,     2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw,       1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw,       2*qy*qz + 2*qx*qw,     1 - 2*qx**2 - 2*qy**2]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)
output_frame_dir = "rendered_frames"
os.makedirs(output_frame_dir, exist_ok=True)

for file_num in tqdm(range(1,2)):  # adjust end as needed
    file_output_dir = os.path.join(output_frame_dir, str(file_num))
    os.makedirs(file_output_dir, exist_ok=True)

    # find all .ply files in ../data/<file_num> and sort by frame index
    names = os.listdir(f'../data/{file_num}')
    pattern_ply = re.compile(r'^[^\.]+\.ply$')
    names_ply = sorted(
        [n for n in names if pattern_ply.match(n)],
        key=lambda x: int(os.path.splitext(x)[0])
    )

    frame_id = 0
    for ply_name in names_ply:
        idx = os.path.splitext(ply_name)[0]

        # — Load point cloud —
        pcd = o3d.io.read_point_cloud(f'../data/{file_num}/{ply_name}')

        # — Load precomputed labels —
        labels = np.load(f'../data/{file_num}/{idx}_labels.npy')\
                   .reshape(-1).astype(int)
        car = json.load(open(f'../data/{file_num}/{idx}_car_state.json'))

        # — Assign colors by label —
        label_to_color = {
            0: (0.5, 0.5, 0.5),  # unknown/background
            1: (0.0, 1.0, 0.0),  # class 1 in green
            2: (1.0, 0.0, 0.0),  # class 2 in red
            # add more mappings if needed
        }
        colors = np.array([label_to_color.get(l, (0.5, 0.5, 0.5)) for l in labels])
        pcd.colors = o3d.utility.Vector3dVector(colors)

        # — Off‐screen render to PNG —
        vis = o3d.visualization.Visualizer()
        vis.create_window(
            window_name=f"SemKITTI-style: Track {file_num} Frame {idx}", visible=True
        )
        vis.add_geometry(pcd)
        ctr = vis.get_view_control()
        
        pos, R = get_pose(car)

        front = R[:, 0]  # or R[:, 1], depending on which column matches car facing
        up = R[:, 2]     # assuming Z-up convention
        lookat = pos + front

        ctr.set_front(-front / np.linalg.norm(front))
        ctr.set_up(up / np.linalg.norm(up))
        ctr.set_lookat(lookat)
        ctr.set_zoom(0.01)
            # camera up is +Z
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        vis.run()
        out_path = os.path.join(file_output_dir, f"frame_{frame_id:04d}.png")
        # vis.capture_screen_image(out_path)
        vis.destroy_window()

        print(f"Saved frame {frame_id} to {out_path}")
        frame_id += 1